In [1]:
from datetime import datetime, timedelta
import pytz
from backend.events_api import get_events as get_events_api
from backend.events_api import transform_event_simple, extract_venue_info, extract_classifications
from flask import request


In [ ]:
classification = 'music,sports'

# Get this week's date range
now = datetime.now(pytz.UTC)
week_start = now - timedelta(days=now.weekday())  # Monday
week_end = week_start + timedelta(days=7)

start_date_time = week_start.strftime("%Y-%m-%dT%H:%M:%SZ")
end_date_time = week_end.strftime("%Y-%m-%dT%H:%M:%SZ")

classification_list = [c.strip() for c in classification.split(',') if c.strip()]

try:
    api_response = get_events_api(
        country_code='CZ',
        city='Prague',
        classification_name=classification_list,
        start_date_time=start_date_time,
        end_date_time=end_date_time,
        size=200,
        sort="date,asc",
        locale="cs",
        save_to_file=False
    )
    
    events_data = api_response.get('_embedded', {}).get('events', [])
    
    result = {
        "events": [transform_event_simple(event) for event in events_data],
        "total": len(events_data),
        "week_start": week_start.strftime("%Y-%m-%d"),
        "week_end": week_end.strftime("%Y-%m-%d"),
        "status": "success"
    }

except Exception as e:
    print(f"Error: {e}")

print(api_response)

{'_links': {'self': {'href': '/discovery/v2/events.json?includeFamily=yes&includeSpellcheck=no&city=Prague&sort=date%2Casc&locale=cs&endDateTime=2025-07-07T19%3A49%3A25Z&unit=miles&startDateTime=2025-06-30T19%3A49%3A25Z&size=200&countryCode=CZ&preferredCountry=us&classificationName=music%2Csports&includeTBA=no&page=0&includeTest=no&includeTBD=no'}}, 'page': {'size': 200, 'totalElements': 0, 'totalPages': 0, 'number': 0}}


In [23]:
# Calculate current date and future date
now = datetime.now(pytz.UTC)
future_date = now + timedelta(days=90)

# Format dates for API (ISO format)
start_date_time = now.strftime("%Y-%m-%dT%H:%M:%SZ")
end_date_time = future_date.strftime("%Y-%m-%dT%H:%M:%SZ")

# Parse classification parameter
classification = 'music,sports'
classification_list = [c.strip() for c in classification.split(',') if c.strip()]

try:
    # Call your Ticketmaster API function
    api_response = get_events_api(
        # country_code='CZ',
        # city='Prague',
        lat_long='50.0755,14.4378',  # Prague center coordinates
        radius=25,  # 25km radius
        # classification_name=classification_list,
        keyword=None,
        start_date_time=start_date_time,
        end_date_time=end_date_time,
        size=min(200, 200),
        sort="date,asc",
        include_tba="no",
        include_tbd="no",
        save_to_file=False,
        page=0
    )
    
    # Extract events from API response
    events_data = api_response.get('_embedded', {}).get('events', [])
    page_info = api_response.get('page', {})
    
    # Log raw events count
    print(f"🎫 Received {len(events_data)} events from Ticketmaster API")
    
    # Transform events data for frontend
    transformed_events = []
    for idx, event in enumerate(events_data):
        transformed_event = {
            'id': event.get('id'),
            'name': event.get('name'),
            'url': event.get('url'),
            'date': event.get('dates', {}).get('start', {}).get('localDate'),
            'time': event.get('dates', {}).get('start', {}).get('localTime'),
            'datetime': event.get('dates', {}).get('start', {}).get('dateTime'),
            'timezone': event.get('dates', {}).get('timezone'),
            'status': event.get('dates', {}).get('status', {}).get('code'),
            'venue': extract_venue_info(event),
            'classifications': extract_classifications(event),
            'price_ranges': event.get('priceRanges', []),
            'images': event.get('images', []),
            'info': event.get('info'),
            'please_note': event.get('pleaseNote')
        }
        transformed_events.append(transformed_event)
        
        # Log each event details
        venue_name = transformed_event['venue'].get('name', 'Unknown venue') if transformed_event['venue'] else 'No venue'
        venue_city = transformed_event['venue'].get('city', 'Unknown city') if transformed_event['venue'] else 'No city'
        
        print (f"  Event {idx + 1}: '{transformed_event['name']}' at {venue_name}, {venue_city} on {transformed_event['date']}")
    
    # Summary logging
    print(f"✅ Successfully transformed {len(transformed_events)} events")
    print(f"📊 Total events available: {page_info.get('totalElements', 0)} across {page_info.get('totalPages', 0)} pages")
    
    # Log first 5 event names for quick reference
    if transformed_events:
        print("🎭 First few events:")
        for i, event in enumerate(transformed_events[:5]):
            print(f"   {i+1}. {event['name']}")
except Exception as e:
    print(f"❌ Error fetching events: {str(e)}")

🎫 Received 117 events from Ticketmaster API
  Event 1: 'Othello' at Pražský hrad, Praha 1 on 2025-07-02
  Event 2: 'VOIVOD - open air, support: EXORCIZPHOBIA' at Cross Club, Praha 7 on 2025-07-03
  Event 3: 'Othello' at Pražský hrad, Praha 1 on 2025-07-03
  Event 4: 'HELL MILITIA, ALTARS ABLAZE, MŮRA' at Eternia, Praha 5 on 2025-07-04
  Event 5: 'Othello' at Pražský hrad, Praha 1 on 2025-07-04
  Event 6: 'Othello' at Pražský hrad, Praha 1 on 2025-07-05
  Event 7: 'Lionel Richie' at O2 Arena, Praha 9 on 2025-07-06
  Event 8: 'Lynyrd Skynyrd' at O2 Arena, Praha 9 on 2025-07-07
  Event 9: 'Othello' at Pražský hrad, Praha 1 on 2025-07-07
  Event 10: 'Othello' at Pražský hrad, Praha 1 on 2025-07-08
  Event 11: 'Ashnikko' at ARCHA+, Praha 1 on 2025-07-09
  Event 12: 'Othello' at Pražský hrad, Praha 1 on 2025-07-09
  Event 13: 'Othello' at Pražský hrad, Praha 1 on 2025-07-10
  Event 14: 'Othello' at Pražský hrad, Praha 1 on 2025-07-11
  Event 15: 'Othello' at Pražský hrad, Praha 1 on 2025-07-